# Awk

## Basics

In [9]:
awk '{ print $1 }' /etc/passwd | head -n 3

root:x:0:0:root:/root:/bin/bash
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin


In [8]:
awk -F: '{ print $1 }' /etc/passwd | head -n 3

root
daemon
bin


In [21]:
cat awk.s

BEGIN { FS=":"; print "User - Shell"; print "----"}
$3 >= 1000 { print $1 " " count ", Shell: " $7; count++ }
END { print "----\ntotal:" NR; print "found:" count }


In [23]:
awk -f awk.s /etc/passwd

User - Shell
----
nobody , Shell: /usr/sbin/nologin
milad 1, Shell: /bin/bash
raven 2, Shell: /bin/bash
libvirt-qemu 3, Shell: /bin/false
----
total:43
found:4


BEGIN, END: will be run once  
FS: The input field separator, a space by default.  
NR: The total number of input records seen so far.  
OFS: The output field separator, a space by default.  

In [24]:
cat awk.s

BEGIN { FS=":"; }
/^m/ { print $1; count++ }
END { print "found:" count }


In [25]:
awk -f awk.s /etc/passwd

man
mail
messagebus
milad
found:4


In [26]:
awk -F':' '{print toupper($1)}' /etc/passwd | head -3

ROOT
DAEMON
BIN


In [27]:
awk -F':' '{print $1 $7}' /etc/passwd | head -3

root/bin/bash
daemon/usr/sbin/nologin
bin/usr/sbin/nologin


In [28]:
awk -F':' '{print $1, $7}' /etc/passwd | head -3

root /bin/bash
daemon /usr/sbin/nologin
bin /usr/sbin/nologin


comma , is field separator.  
The default separator is space.  
we can change it using OSF variable.

In [57]:
awk -F':' 'BEGIN { OFS=":" } { count++; print count, $1, $7, rand() }' /etc/passwd | head -3

1:root:/bin/bash:0.237788
2:daemon:/usr/sbin/nologin:0.291066
3:bin:/usr/sbin/nologin:0.845814


## And

In [62]:
last | grep milad | grep pts/7

milad    pts/7        :0               Tue May  2 20:32 - 20:33  (00:00)


In [80]:
last | awk "(/milad/ && /pts\/7/)"

milad    pts/7        :0               Tue May  2 20:32 - 20:33  (00:00)


## Or

In [64]:
last | grep milad | grep -e pts/6 -e  pts/7

milad    pts/6        :0.0             Sun May 21 22:40 - 01:21  (02:40)
milad    pts/6        :0               Fri May  5 23:04 - 23:05  (00:00)
milad    pts/7        :0               Tue May  2 20:32 - 20:33  (00:00)
milad    pts/6        :0               Tue May  2 20:32 - 20:34  (00:01)
milad    pts/6        :0               Tue May  2 20:05 - 20:05  (00:00)
milad    pts/6        :0.0             Mon May  1 22:23 - 00:10  (01:47)


In [65]:
last | grep milad | grep -E "pts/6|pts/7"

milad    pts/6        :0.0             Sun May 21 22:40 - 01:21  (02:40)
milad    pts/6        :0               Fri May  5 23:04 - 23:05  (00:00)
milad    pts/7        :0               Tue May  2 20:32 - 20:33  (00:00)
milad    pts/6        :0               Tue May  2 20:32 - 20:34  (00:01)
milad    pts/6        :0               Tue May  2 20:05 - 20:05  (00:00)
milad    pts/6        :0.0             Mon May  1 22:23 - 00:10  (01:47)


In [76]:
last | awk "(/milad/ && (/pts\/6/||/pts\/7/))"

milad    pts/6        :0.0             Sun May 21 22:40 - 01:21  (02:40)
milad    pts/6        :0               Fri May  5 23:04 - 23:05  (00:00)
milad    pts/7        :0               Tue May  2 20:32 - 20:33  (00:00)
milad    pts/6        :0               Tue May  2 20:32 - 20:34  (00:01)
milad    pts/6        :0               Tue May  2 20:05 - 20:05  (00:00)
milad    pts/6        :0.0             Mon May  1 22:23 - 00:10  (01:47)


In [81]:
lastlog | grep -v "Never"

Username         Port     From             Latest
milad            tty1                      Fri May 26 11:28:43 +0430 2017


In [85]:
lastlog | awk "!(/Never/)"

Username         Port     From             Latest
milad            tty1                      Fri May 26 11:28:43 +0430 2017


In [5]:
awk "/milad/ && /syslog/" /etc/group

adm:x:4:syslog,milad


In [29]:
awk '(/milad/ || /syslog/) && !(/milad$/)' /etc/group

syslog:x:108:
milad:x:1000:


In [25]:
awk -F ':' '(/milad/ || /syslog/) && !(/milad$/) && $3 < 1000 { print $1 }' /etc/group

syslog


In [50]:
awk 'BEGIN {FS=":"; print "- - - - -"} (/milad/ || /syslog/) && !(/milad$/) && $3 < 1000 { cnt++; print cnt":"$1 }\
END { print "* * * * *\nTotal rows:" NR; print "Found:" cnt "\n" "Fields:" NF}' /etc/group

- - - - -
1:syslog
* * * * *
Total rows:71
Found:1
Fields:4


In [54]:
grep adm: /etc/group

adm:x:4:syslog,milad


In [7]:
cat awk.s

BEGIN { FS=":"; print "------"; count=0;}
$4 ~ $1{
 count++; print $1 
}
END { print "------"; print "Number:", count }



In [8]:
awk -f awk.s /etc/group

------
milad
raven
------
Number: 2


In [9]:
cat awk.s # same as above script

BEGIN { FS=":"; print "------"; count=0;}
{
 if($4 ~ $1){
 count++; print $1
 }
}
END { print "------"; print "Number:", count }


In [10]:
cat txt

hi this
is a
sample file

here is
somethig else
separated with
two new line


In [13]:
awk 'BEGIN {RS="\n\n"} $0 ~ /sep/{ print }' txt

here is
somethig else
separated with
two new line



RS: Record separator, default is \n

In [6]:
awk '/sep/{ print }' txt

separated with


In [21]:
awk -F":" '$4 ~ /milad/ { if ($1 != "milad") a[$1]=gsub(",", $4)} END { for ( i in a) { print i, ":", ++a[i] } }' \
/etc/group # find number of users each group has

docker : 1
wireshark : 1
lpadmin : 1
dip : 1
sambashare : 1
plugdev : 1
sudo : 1
cdrom : 1
adm : 2


- - -
## Lecture notes

- All lecture notes are available at github in `ipynb` format: [https://github.com/ravexina/linux-notes](https://github.com/ravexina/linux-notes). 


- There are `html` exports of project available at: [https://ravexina.github.io/linux-notes](https://ravexina.github.io/linux-notes).

## License
<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" />
</a>

<span property="dct:title">Linux Notes</span> by 
<a href="http://github.com/ravexina/Linux-Notes" property="cc:attributionName" rel="cc:attributionURL">Milad As (Ravexina)</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

- - -

<a rel="license" href="https://gitlab.com/ravexina/"><img alt="ravexina's gitlab" style="display:inline;border-width:0;height:30px;float:left;" src="img/logo/logo-square.png" /></a> 

<a rel="license" href="https://github.com/ravexina/"><img alt="ravexina's github" style="float:right;display:inline;border-width:0;height:30px;" src="img/logo/GitHub_Logo.png" /></a>